In [2]:
import os
import re
from bs4 import BeautifulSoup

In [3]:
ls

003_0008.html   html2xml.ipynb


In [4]:
file = '003_0008.html'

In [5]:
with open(file,'r') as f:
    lines = f.readlines()
    f.seek(0)
    raw = f.read()

In [6]:
raw[:100]

'<div class="container">\n<div class="book-utils">\n<div id="qrcode"></div>\n<a href="/Jan_Barsceuski/Za'

In [7]:
lines[:30]

['<div class="container">\n',
 '<div class="book-utils">\n',
 '<div id="qrcode"></div>\n',
 '<a href="/Jan_Barsceuski/Zaroslaje_voziera.epub">epub</a>\n',
 '<div>\xa0</div>\n',
 '<div style="vertical-align: middle">\n',
 '<label class="switch">\n',
 '<input id="dictSwitch" type="checkbox"/>\n',
 '<span class="slider round"></span>\n',
 '</label>\n',
 '</div>\n',
 '<div id="nodictLabel">Падключыць<br/>слоўнікі</div>\n',
 '<div id="dictLabel" style="display: none">Націскайце на<br/>словы ў тэксце</div>\n',
 '</div>\n',
 '<!-- HEADER_FIELD Authors: Ян Баршчэўскі -->\n',
 '<!-- HEADER_FIELD CreationYear: 1842 -->\n',
 '<!-- HEADER_FIELD Edition: Ян Баршчэўскі. Выбраныя творы. Менск, МФ «Беларускі кнігазбор», 1998. -->\n',
 '<!-- HEADER_FIELD FirstPublicationYear: 1843 -->\n',
 '<!-- HEADER_FIELD LangOrig: pol -->\n',
 '<!-- HEADER_FIELD Pravapis: A1957 -->\n',
 '<!-- HEADER_FIELD PublicationYear: 1998 -->\n',
 '<!-- HEADER_FIELD SectionAuthor: Балады -->\n',
 '<!-- HEADER_FIELD StyleGenre:

In [8]:
def meta(file): 
    with open(file,'r') as f:
        raw = f.read()
        
    metadict = {}
    ptrn = re.compile('(?<=<!-- HEADER_FIELD ).*(?= -->)')
    meta = re.findall(ptrn, raw)

    if meta: 
        for m in meta:
            metadict[m.split(':')[0]] = m.split(':')[1].lstrip()
            
    return metadict

In [9]:
meta(file)

{'Authors': 'Ян Баршчэўскі',
 'CreationYear': '1842',
 'Edition': 'Ян Баршчэўскі. Выбраныя творы. Менск, МФ «Беларускі кнігазбор», 1998.',
 'FirstPublicationYear': '1843',
 'LangOrig': 'pol',
 'Pravapis': 'A1957',
 'PublicationYear': '1998',
 'SectionAuthor': 'Балады',
 'StyleGenre': 'мастацкі/балада',
 'Title': 'Зарослае возера',
 'Title2': 'Балада з народных паданняў',
 'Translation': 'Уладзімір Мархель'}

In [10]:
meta(file)['Authors']

'Ян Баршчэўскі'

In [11]:
def lines(raw):
    soup = BeautifulSoup(raw,'html.parser')
    text = soup.find('div', {'class': 'POETRY'})
    lines = [l.getText().strip() for l in text.find_all('p')]
    
    return lines

def struct(raw):
    structure = []
    stanza = []

    for i,line in enumerate(getlines(raw)):

        if i == 0 and not line:
            continue
        
        if line:
            stanza.append(line)

        else:
            structure.append(stanza)
            stanza = []

    return structure

In [12]:
struct(raw)

NameError: name 'getlines' is not defined

In [13]:
newtree = BeautifulSoup('', 'xml')

newtree.append(newtree.new_tag('TEI'))

head = newtree.new_tag('teiHeader')
newtree.TEI.append(head)

author = newtree.new_tag('author')
author.string = meta(file)['Authors']
newtree.teiHeader.append(author)


body = newtree.new_tag('body')
newtree.TEI.append(body)


for stanza in struct(raw):
    if len(stanza) == 4:
        st = newtree.new_tag('lg', type = 'quatrain')
        
    else:
        st = newtree.new_tag('lg')
    
    for l in stanza:
        line = newtree.new_tag('l')
        line.string = l
        st.append(line)
        
    newtree.body.append(st)
        
newtree

NameError: name 'getlines' is not defined

In [14]:
newtree.TEI.teiHeader

<teiHeader><author>Ян Баршчэўскі</author></teiHeader>

In [15]:
newtree.TEI.body

<body/>

In [16]:
newtree.TEI.body.find_all('l')

[]